In [ ]:
SYFT_VERSION = ">=0.8.2.b0,<0.9"
package_string = f'"syft{SYFT_VERSION}"'
# %pip install {package_string} -f https://whls.blob.core.windows.net/unstable/index.html -q

In [ ]:
import syft as sy
import numpy as np
sy.requires(SYFT_VERSION)

In [ ]:
node = sy.orchestra.launch(name="test-domain-1", port="auto", dev_mode=True)

In [ ]:
root_client = node.login(email="info@openmined.org", password="changethis")

In [ ]:
sample_data = np.array([6., 34, 78, 91.3, 21.5])
mock_sample_data = np.array([7., 54, 88, 11, 28.3])

In [ ]:
dataset = sy.Dataset(name="My Sample Dataset")
asset = sy.Asset(name="Sample Data")
asset.set_obj(sample_data)
asset.set_mock(mock_sample_data, mock_is_real=False)
asset.set_shape(sample_data.shape)
dataset.add_asset(asset)

In [ ]:
root_client.upload_dataset(dataset)

In [ ]:
root_client.register(name="Sheldon Cooper",
                             email="sheldon@caltech.edu",
                             password="abc123",
                             password_verify="abc123",
                             institution="Caltech",
                             website="https://www.caltech.edu/")

In [ ]:
ds_client = node.login(email="sheldon@caltech.edu", password="abc123")

In [ ]:
ds_client.datasets

In [ ]:
datasets = ds_client.datasets.search(name="My Sample Dataset")

In [ ]:
assert len(datasets) == 1
dataset_ptr = datasets[0]
dataset_ptr

In [ ]:
# The Data Scientist does some basic statistics function on the object
@sy.syft_function(input_policy=sy.ExactMatch(data=dataset_ptr.assets[0]),
                  output_policy=sy.SingleExecutionExactOutput())
def calculate_sum(data):
    import opendp.prelude as dp
    dp.enable_features("contrib")
    from opendp.measurements import make_laplace
    
    #compute sum
    res = data.sum()
    base_lap = dp.m.make_base_laplace(
        dp.atom_domain(T=float),
        dp.absolute_distance(T=float),
        scale=10.
    )
    noise_result = base_lap(res)
    return noise_result

In [ ]:
new_project = sy.Project(
    name="My Cool  Project",
    description="""Hi, I want to do statistics operations(like sum, mean) on your private data.""",
    members=[ds_client],
)
new_project

In [ ]:
new_project.create_code_request(calculate_sum, ds_client)

In [ ]:
project = new_project.start()

project 

In [ ]:
assert isinstance(project, sy.service.project.project.Project)

In [ ]:
assert len(project.events) ==1
assert isinstance(project.events[0], sy.service.project.project.ProjectRequest)

In [ ]:
# The Data Scientist would like to submit another code request after the start of the project
@sy.syft_function(input_policy=sy.ExactMatch(data=dataset_ptr.assets[0]),
                  output_policy=sy.SingleExecutionExactOutput())
def calculate_mean(data):
    import opendp.prelude as dp
    dp.enable_features("contrib")
    from opendp.measurements import make_laplace
    
    #compute mean
    mean = data.mean()
    base_lap = dp.m.make_base_laplace(
        dp.atom_domain(T=float),
        dp.absolute_distance(T=float),
        scale=10.
    )
    noise_result = base_lap(mean)
    return noise_result

In [ ]:
res = project.create_code_request(calculate_mean ,ds_client)
res

In [ ]:
assert isinstance(res, sy.SyftSuccess)

In [ ]:
assert len(project.events) == 2
assert isinstance(project.events[1], sy.service.project.project.ProjectRequest)

In [ ]:
# The Domain Owner retrieves by name or uid for approval
root_client_project = root_client.projects.get_by_uid(project.id)
assert isinstance(root_client_project, sy.service.project.project.Project)

In [ ]:
root_client_project.requests
assert len(root_client_project.requests) == 2

In [ ]:
# The Data Owner approves both requests
request_1 = root_client_project.requests[0].approve()
request_1

In [ ]:
assert isinstance(request_1, sy.SyftSuccess)

In [ ]:
request_2 = root_client_project.requests[1].approve()
request_2

In [ ]:
assert isinstance(request_2, sy.SyftSuccess)

In [ ]:
ds_client.datasets

In [ ]:
sum_ptr = ds_client.code.calculate_sum(data=dataset_ptr.assets[0])

In [ ]:
sum_ptr

In [ ]:
sum_result = sum_ptr.get()

In [ ]:
assert isinstance(sum_result, float)

In [ ]:
mean_ptr = ds_client.code.calculate_mean(data=dataset_ptr.assets[0])

In [ ]:
mean_result = mean_ptr.get()

In [ ]:
assert isinstance(mean_result,float)

In [ ]:
if node.node_type.value == "python":
    node.land()